In [1]:
import json
import pandas as pd
import numpy as np
from random import choice
from sklearn.impute import SimpleImputer
file = open("case study.txt","r")
df = pd.read_csv("C://Users/HP/OneDrive/Desktop/Screening Test - DS/iris.csv")
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [2]:
data = file.read()
dict1=json.loads(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
dict1

{'session_name': 'test',
 'session_description': 'test',
 'design_state_data': {'session_info': {'project_id': '1',
   'experiment_id': 'kkkk-11',
   'dataset': 'iris_modified.csv',
   'session_name': 'test',
   'session_description': 'test'},
  'target': {'prediction_type': 'Regression',
   'target': 'petal_width',
   'type': 'regression',
   'partitioning': True},
  'train': {'policy': 'Split the dataset',
   'time_variable': 'sepal_length',
   'sampling_method': 'No sampling(whole data)',
   'split': 'Randomly',
   'k_fold': False,
   'train_ratio': 0,
   'random_seed': 0},
  'metrics': {'optomize_model_hyperparameters_for': 'AUC',
   'optimize_threshold_for': 'F1 Score',
   'compute_lift_at': 0,
   'cost_matrix_gain_for_true_prediction_true_result': 1,
   'cost_matrix_gain_for_true_prediction_false_result': 0,
   'cost_matrix_gain_for_false_prediction_true_result': 0,
   'cost_matrix_gain_for_false_prediction_false_result': 0},
  'feature_handling': {'sepal_length': {'feature_name'

In [49]:
y = dict1["design_state_data"]["target"]["target"]
y = df[y]
feature = []
for feature1 in dict1["design_state_data"]["feature_handling"]:
    feature.append(feature1)
missing_value = []
for x in feature:
    if ("impute_with" in dict1["design_state_data"]["feature_handling"][x]["feature_details"]) and (dict1["design_state_data"]["feature_handling"][x]["feature_details"]['impute_with'] == "Average of values"):
        impute_value = dict1["design_state_data"]["feature_handling"][x]["feature_details"]['impute_value']
        Average = df[x].mean()
        df[x] = df[x].replace(int(impute_value),Average)
    elif ("impute_with" in dict1["design_state_data"]["feature_handling"][x]["feature_details"]) and (dict1["design_state_data"]["feature_handling"][x]["feature_details"]["impute_with"] == "custom"):
        impute_value = dict1["design_state_data"]["feature_handling"][x]["feature_details"]['impute_value']
        Custom = df[x].median()
        df[x] = df[x].replace(int(impute_value),Custom)
    else:
        Custom = df[x].mode()
        Custom1 = []
        for i in Custom:
            Custom1.append(i)
        Custom2 = choice(Custom1)
        df[x] = df[x].replace(np.nan,Custom2)
x = df.drop(dict1["design_state_data"]["target"]["target"],axis=1)
col = []
for i in x.columns:
    col.append(i)
from sklearn.preprocessing import LabelEncoder
for i in col:
    if (x[i]).dtype == "object":
        LE = LabelEncoder()
        x[i] = LE.fit_transform(x[i])
    else:
        continue
for i in col:
    if x[i].corr(y) == 1:
        df.drop(i,axis = 1,inplace=True)
    else:
        continue
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
SC = StandardScaler()
fea = []
tar = dict1["design_state_data"]["target"]["target"]
for i in col:
    if ((df[i]).dtype != "object"):
        fea.append(i)
    else:
        continue
df[tar] = SC.fit_transform(df[[tar]])
x[fea] = SC.fit_transform(x[fea])
y = df[tar]
no_of_columns = len(x.columns)
if no_of_columns > int(dict1["design_state_data"]["feature_reduction"]["num_of_features_to_keep"]):
    pca = PCA(n_components= int(dict1["design_state_data"]["feature_reduction"]["num_of_features_to_keep"]))
    x[fea] = pca.fit_transform(x[fea])
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.25,random_state=2)
from sklearn import tree,linear_model,ensemble,neighbors,svm
from sklearn.metrics import r2_score,accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import GridSearchCV
def regg(model):
    model.fit(xtrain,ytrain)
    pred = model.predict(xtest)
    train = model.score(xtrain,ytrain)
    test = model.score(xtest,ytest)
    print(f"train : {train}\ntest : {test}")
    print(f"r2_score : {r2_score(ytest,pred)}")
    print("-"*100)
def classification(model):
    model.fit(xtrain,ytrain)
    pred = model.predict(xtest)
    train = model.score(xtrain,ytrain)
    test = model.score(xtest,ytest)
    print(f"train : {train}\ntest : {test}")
    print(f"accuracy_score : {accuracy_score(ytest,pred)}")
    print(f"classification_report :\n {classification_report(ytest,pred)}")
    print(f"confusion_matrix :\n {confusion_matrix(ytest,pred)}")
    print("-"*100)
algorithms = []
for i in dict1["design_state_data"]["algorithms"].keys():
    algorithms.append(i)
if (dict1["design_state_data"]["target"]["prediction_type"] == "Regression") or (dict1["design_state_data"]["target"]["prediction_type"] == "regression"):
    if "LinearRegression" in algorithms:
        print("LinearRegression")
        regg(linear_model.LinearRegression())
    if "DecisionTreeRegressor" in algorithms:
        print("DecisionTreeRegressor")
        regg(tree.DecisionTreeRegressor())
    if "RandomForestRegressor" in algorithms:
        print("RandomForestRegressor")
        regg(ensemble.RandomForestRegressor())
    if "GBTRegressor" in algorithms:
        print("GBTRegressor")
        regg(ensemble.GradientBoostingRegressor())
    if "neural_network" in algorithms:
        import tensorflow as tf
        from tensorflow.keras import Sequential 
        from tensorflow.keras.layers import Dense
    ann = Sequential()
    hidden_layer = dict1["design_state_data"]["algorithms"]["neural_network"]["hidden_layer_sizes"]
    hidden = choice(hidden_layer)
    ann.add(Dense(units = hidden,activation = "relu"))
    ann.add(Dense(units = 1))
    solver = dict1["design_state_data"]["algorithms"]["neural_network"]["solver"]
    ann.compile(optimizer = solver,loss = "mse")
    from tensorflow.keras.callbacks import EarlyStopping
    earlystop = EarlyStopping(monitor = "val_loss",mode = "min",verbose = 1,patience = 25)
    ann.fit(xtrain,ytrain,epochs = 500,validation_data = (xtest,ytest),callbacks = [earlystop])
    ypred = ann.predict(xtest)
    print("-"*100)
    print("r2_score:",r2_score(ytest,ypred))
if (dict1["design_state_data"]["target"]["prediction_type"] == "Classification") or (dict1["design_state_data"]["target"]["prediction_type"] == "classification"):
    if "LogisticRegression" in algorithms:
        print("LogisticRegression")
        regg(linear_model.LogisticRegression())
    if "DecisionTreeClassifier" in algorithms:
        print("DecisionTreeClassifier")
        regg(tree.DecisionTreeClassifier())
    if "RandomForestClassifier" in algorithms:
        print("RandomForestClassifier")
        regg(ensemble.RandomForestClassifier())
    if "GBTClassifier" in algorithms:
        print("GBTClassifier")
        regg(ensemble.GradientBoostingClassifier())
    if "neural_network" in algorithms:
        import tensorflow as tf
        from tensorflow.keras import Sequential 
        from tensorflow.keras.layers import Dense
    g = []
    for i in df["species"].unique():
        g.append(i)
    no_of_catogories = len(g)
    if no_of_catogories > 2:
        ann = Sequential()
        hidden_layer = dict1["design_state_data"]["algorithms"]["neural_network"]["hidden_layer_sizes"]
        hidden = choice(hidden_layer)
        ann.add(Dense(units = hidden,activation = "relu"))
        ann.add(Dense(units = no_of_catogories,activation="softmax"))
        solver = dict1["design_state_data"]["algorithms"]["neural_network"]["solver"]
        ann.compile(optimizer = solver,loss = "sparse_categorical_crossentropy")
        from tensorflow.keras.callbacks import EarlyStopping
        earlystop = EarlyStopping(monitor = "val_loss",mode = "min",verbose = 1,patience = 25)
        ann.fit(xtrain,ytrain,epochs = 500,validation_data = (xtest,ytest),callbacks = [earlystop])
        ypred = ann.predict(xtest)
        print("-"*100)
        print("r2_score:",r2_score(ytest,ypred))
    else:
        ann = Sequential()
        hidden_layer = dict1["design_state_data"]["algorithms"]["neural_network"]["hidden_layer_sizes"]
        hidden = choice(hidden_layer)
        ann.add(Dense(units = hidden,activation = "relu"))
        ann.add(Dense(units = 1,activation="sigmoid"))
        solver = dict1["design_state_data"]["algorithms"]["neural_network"]["solver"]
        ann.compile(optimizer = solver,loss = "binary_crossentropy")
        from tensorflow.keras.callbacks import EarlyStopping
        earlystop = EarlyStopping(monitor = "val_loss",mode = "min",verbose = 1,patience = 25)
        ann.fit(xtrain,ytrain,epochs = 500,validation_data = (xtest,ytest),callbacks = [earlystop])
        ypred = ann.predict(xtest)
        print("-"*100)
        print("r2_score:",r2_score(ytest,ypred))
        

LinearRegression
train : 0.9508373303604417
test : 0.9519107849759174
r2_score : 0.9519107849759174
----------------------------------------------------------------------------------------------------
DecisionTreeRegressor
train : 0.9999213932485223
test : 0.8993365787914348
r2_score : 0.8993365787914348
----------------------------------------------------------------------------------------------------
RandomForestRegressor
train : 0.9913220398976054
test : 0.943182427616215
r2_score : 0.943182427616215
----------------------------------------------------------------------------------------------------
GBTRegressor
train : 0.9887338900447172
test : 0.941825522635523
r2_score : 0.941825522635523
----------------------------------------------------------------------------------------------------
Epoch 1/500
4/4 [==============================] - 0s 42ms/step - loss: 1.1475 - val_loss: 1.0169
Epoch 2/500
4/4 [==============================] - 0s 10ms/step - loss: 0.9141 - val_loss: 0.820